# Query Recommender

In [141]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.metrics.pairwise import euclidean_distances

# Eingabedaten erzeugen

In [142]:
inputdata = pd.read_pickle("../data/recodata.pkl")

In [143]:
inputdata = pd.DataFrame([[0]*len(inputdata.columns)], columns=inputdata.columns)
inputdata

,yearpublished,playingtime,age,sub_AbstractGames,sub_CustomizableGames,sub_FamilyGames,sub_PartyGames,sub_StrategyGames,sub_ThematicGames,sub_Wargames,...,mec_TurnOrder:Random,mec_Slide/Push,mec_Acting,mec_Ordering,mec_PassedActionToken,mec_PiecesasMap,mec_Ratio/CombatResultsTable,mec_RelativeMovement,mec_Singing,mec_MinimapResolution
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
#inputdata.columns.tolist()

In [145]:
inputdata["yearpublished"] = 2020
inputdata["playingtime"] = 60
inputdata["age"] = 10
inputdata["sub_FamilyGames"] = 1
for x in ["cat_CardGame", "cat_Dice", "cat_Animals", "mec_DiceRolling", "mec_ModularBoard"]:
        inputdata[x]=1

In [146]:
inputdata

,yearpublished,playingtime,age,sub_AbstractGames,sub_CustomizableGames,sub_FamilyGames,sub_PartyGames,sub_StrategyGames,sub_ThematicGames,sub_Wargames,...,mec_TurnOrder:Random,mec_Slide/Push,mec_Acting,mec_Ordering,mec_PassedActionToken,mec_PiecesasMap,mec_Ratio/CombatResultsTable,mec_RelativeMovement,mec_Singing,mec_MinimapResolution
0,2020,60,10,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
# Test mit echten Werten
# inputdata = pd.read_pickle("../data/recotestdata.pkl")
# 0 = BRASS
# inputdata = inputdata.loc[[0],:].copy()
# inputdata

# Eingabedaten transformieren

In [148]:
pca_cat = joblib.load('../data/pca_cat.pkl')
ncomp = pca_cat.n_components_ 
categories_pca = pd.DataFrame(pca_cat.transform(inputdata[[x for x in inputdata.columns if x[:4]=="cat_"]]), columns=["catp_" + str(x) for x in range(ncomp)])

In [149]:
categories_pca

,catp_0,catp_1,catp_2,catp_3,catp_4,catp_5,catp_6,catp_7,catp_8,catp_9,catp_10,catp_11,catp_12,catp_13,catp_14,catp_15,catp_16,catp_17,catp_18,catp_19
0,-0.140158,0.771225,-0.226547,-0.054456,-0.077794,-0.100844,-0.573625,-0.147317,-0.521231,-0.057255,0.017202,0.369378,0.418195,0.382516,-0.136659,-0.264484,-0.196215,0.469878,0.404962,-0.026551


In [150]:
pca_mec = joblib.load('../data/pca_mec.pkl')

In [151]:
ncomp = pca_mec.n_components_ 

In [152]:
mechanic_pca = pd.DataFrame(pca_mec.transform(inputdata[[x for x in inputdata.columns if x[:4]=="mec_"]]), columns=["mecp_" + str(x) for x in range(ncomp)])

In [153]:
mechanic_pca

,mecp_0,mecp_1,mecp_2,mecp_3,mecp_4,mecp_5,mecp_6,mecp_7,mecp_8,mecp_9,mecp_10,mecp_11,mecp_12,mecp_13,mecp_14,mecp_15,mecp_16,mecp_17,mecp_18,mecp_19
0,0.122441,-0.620022,-0.141741,-0.762233,0.143688,-0.063771,-0.062315,-0.184435,-0.415291,-0.057995,-0.225565,0.160119,0.020741,0.423291,-0.141717,0.041275,-0.31988,0.078468,-0.176082,0.180832


### StandardScaler laden

In [154]:
allfeat = ['yearpublished', 'playingtime', 'age', 'sub_AbstractGames', 'sub_CustomizableGames', 'sub_FamilyGames', 
           'sub_PartyGames', 'sub_StrategyGames', 'sub_ThematicGames', 'sub_Wargames']
for x in allfeat:
    scaler = joblib.load(f'../data/scaler_{x}.pkl')
    inputdata[x] = scaler.transform(inputdata[x].values.reshape(-1, 1))

In [155]:
inputdata

,yearpublished,playingtime,age,sub_AbstractGames,sub_CustomizableGames,sub_FamilyGames,sub_PartyGames,sub_StrategyGames,sub_ThematicGames,sub_Wargames,...,mec_TurnOrder:Random,mec_Slide/Push,mec_Acting,mec_Ordering,mec_PassedActionToken,mec_PiecesasMap,mec_Ratio/CombatResultsTable,mec_RelativeMovement,mec_Singing,mec_MinimapResolution
0,0.081432,-0.460804,-0.992451,-0.181818,-0.149983,1.927947,-0.214535,-1.327566,-0.496873,-0.214535,...,0,0,0,0,0,0,0,0,0,0


In [156]:
y_test = inputdata[allfeat].copy()

In [157]:
y_test = pd.concat([y_test,categories_pca], axis=1)
y_test = pd.concat([y_test,mechanic_pca], axis=1)

In [158]:
y_test

,yearpublished,playingtime,age,sub_AbstractGames,sub_CustomizableGames,sub_FamilyGames,sub_PartyGames,sub_StrategyGames,sub_ThematicGames,sub_Wargames,...,mecp_10,mecp_11,mecp_12,mecp_13,mecp_14,mecp_15,mecp_16,mecp_17,mecp_18,mecp_19
0,0.081432,-0.460804,-0.992451,-0.181818,-0.149983,1.927947,-0.214535,-1.327566,-0.496873,-0.214535,...,-0.225565,0.160119,0.020741,0.423291,-0.141717,0.041275,-0.31988,0.078468,-0.176082,0.180832


# Distanzen berechnen

In [159]:
x_train = joblib.load('../data/x_train.pkl')

In [160]:
x_train

,yearpublished,playingtime,age,sub_AbstractGames,sub_CustomizableGames,sub_FamilyGames,sub_PartyGames,sub_StrategyGames,sub_ThematicGames,sub_Wargames,...,mecp_10,mecp_11,mecp_12,mecp_13,mecp_14,mecp_15,mecp_16,mecp_17,mecp_18,mecp_19
0,0.070905,0.198523,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,-0.496873,-0.214535,...,-0.248174,-0.903544,0.589259,1.093204,0.352583,-0.606404,0.293997,-0.045777,-0.254134,0.483752
1,0.055116,-0.460804,0.452866,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,2.012587,-0.214535,...,1.009282,-0.349837,0.031275,-0.545157,0.112957,-0.081386,0.402578,0.304836,-0.065632,-0.237580
2,0.065642,0.198523,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,2.012587,-0.214535,...,-0.549885,0.744154,0.982925,-0.786259,0.153607,0.032117,1.202966,0.328467,0.612550,0.261150
3,0.086695,0.528187,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,-0.496873,-0.214535,...,-0.286998,-0.097076,-0.037329,-0.343061,0.532346,-0.051294,-0.284268,-0.022776,-0.193163,0.089833
4,0.065642,4.154487,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,2.012587,-0.214535,...,-0.031798,0.332129,-0.329378,0.259277,-0.241409,0.046328,0.169045,0.118251,-1.085089,0.142976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.097221,-0.131140,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,-0.496873,-0.214535,...,0.099418,-0.327680,-0.340473,0.011368,0.669996,-0.342519,0.019506,-0.004249,-0.171493,0.240011
496,0.060379,-0.625636,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,-0.496873,-0.214535,...,0.330123,-0.067240,-0.320883,0.280889,-0.110249,0.099180,0.150194,-0.239745,0.113378,0.111486
497,0.097221,-0.131140,0.934639,-0.181818,-0.149983,-0.518686,-0.214535,0.753258,-0.496873,-0.214535,...,-0.104443,0.040413,0.119826,0.051949,0.238471,0.005678,-0.331433,0.665429,-0.024894,-0.219595
498,0.049853,-0.680580,-0.992451,-0.181818,-0.149983,1.927947,-0.214535,-1.327566,-0.496873,-0.214535,...,-0.309769,0.132605,-0.177347,-0.090203,0.207936,0.518857,0.197350,-0.114309,-0.090921,-0.197803


In [161]:
recos = pd.DataFrame(euclidean_distances( x_train, y_test))

# Restliche Daten joinen

In [162]:
bgdata = pd.read_pickle("../data/bg_data20240412.pkl")

In [163]:
recos = pd.concat([recos, bgdata["name"]], axis=1)

In [164]:
recos.sort_values(0)[:40]

,0,name,name
405,1.929408,Twice as Clever!,"['Dobbel zo Clever', 'Doppelt so Clever', 'Dup..."
171,2.167094,That's Pretty Clever!,"['Clever', 'Ganz Schön Clever', 'Optimus', 'Pr..."
115,2.179849,SCOUT,"['Cirkus', 'SCOUT', 'Scout!', 'スカウト！', '马戏星探',..."
494,2.229092,Point Salad,"['Ensalada de puntos', 'Insalata di Punti', 'P..."
350,2.410069,Cat in the Box: Deluxe Edition,"['Cat in the Box', 'Cat in the Box: Deluxe Edi..."
334,2.447881,For Sale,"['EladLak!', 'For Sale', 'Garage (گاراج)', 'Na..."
301,2.473692,Dice Throne: Season Two – Battle Chest,"['Dice Throne: Saison Deux – Battle Chest', 'D..."
46,2.497619,Crokinole,"['Carquiñol', 'Carquinyoli', 'Crokinole', 'Cro..."
329,2.522117,Lost Cities,"['Les Cités Perdues', 'Le Città Perdute', 'Elv..."
326,2.524745,Skull King,"[""Grandpa Beck's Scheming and Skulking"", ""Gran..."


In [165]:
bgdata.loc[10,["name", "boardgamemechanic", "boardgamecategory"]]["boardgamemechanic"]

"['Action Retrieval', 'Area Majority / Influence', 'Automatic Resource Growth', 'Campaign / Battle Card Driven', 'Cooperative Game', 'Hand Management', 'Modular Board', 'Set Collection', 'Simultaneous Action Selection', 'Solo / Solitaire Game', 'Tags', 'Variable Player Powers', 'Variable Set-up']"